In [9]:
import geopandas as gpd
import pandas as pd
import os
from pathlib import Path
import fiona
import re
from dotenv import load_dotenv

In [10]:
lcz_data = {
    'LCZ' : list(range(1, 18)),
    'lcz_code': [
        'LCZ 1', 'LCZ 2', 'LCZ 3', 'LCZ 4', 'LCZ 5', 'LCZ 6', 'LCZ 7', 'LCZ 8', 'LCZ 9', 'LCZ 10',
        'LCZ 11 (A)', 'LCZ 12 (B)', 'LCZ 13 (C)', 'LCZ 14 (D)', 'LCZ 15 (E)', 'LCZ 16 (F)', 'LCZ 17 (G)'
    ],
    'description': [
        'Compact highrise', 'Compact midrise', 'Compact lowrise', 'Open highrise', 'Open midrise',
        'Open lowrise', 'Lightweight low-rise', 'Large lowrise', 'Sparsely built', 'Heavy Industry',
        'Dense trees', 'Scattered trees', 'Bush, scrub', 'Low plants', 'Bare rock or paved',
        'Bare soil or sand', 'Water'
    ],
    'color': [
        '#910613', '#D9081C', '#FF0A22', '#C54F1E', '#FF6628', '#FF985E', '#FDED3F', '#BBBBBB',
        '#FFCBAB', '#565656', '#006A18', '#00A926', '#628432', '#B5DA7F', '#000000', '#FCF7B1',
        '#656BFA'
    ]
}

lcz_label_df = pd.DataFrame(lcz_data)

print(lcz_label_df)

    LCZ    lcz_code           description    color
0     1       LCZ 1      Compact highrise  #910613
1     2       LCZ 2       Compact midrise  #D9081C
2     3       LCZ 3       Compact lowrise  #FF0A22
3     4       LCZ 4         Open highrise  #C54F1E
4     5       LCZ 5          Open midrise  #FF6628
5     6       LCZ 6          Open lowrise  #FF985E
6     7       LCZ 7  Lightweight low-rise  #FDED3F
7     8       LCZ 8         Large lowrise  #BBBBBB
8     9       LCZ 9        Sparsely built  #FFCBAB
9    10      LCZ 10        Heavy Industry  #565656
10   11  LCZ 11 (A)           Dense trees  #006A18
11   12  LCZ 12 (B)       Scattered trees  #00A926
12   13  LCZ 13 (C)           Bush, scrub  #628432
13   14  LCZ 14 (D)            Low plants  #B5DA7F
14   15  LCZ 15 (E)    Bare rock or paved  #000000
15   16  LCZ 16 (F)     Bare soil or sand  #FCF7B1
16   17  LCZ 17 (G)                 Water  #656BFA


In [11]:
def process_grid_data_by_city(grid_path, csv_path, lcz_path, output_dir, id_field='id', exclude_layers=None, lcz_label_df = lcz_label_df):
    """
    Process grid data from a GeoPackage and CSV for multiple layers,
    creating separate GeoJSON files for each city.
    
    Parameters:
    -----------
    grid_path : str
        Path to the GeoPackage file containing the grid layers
    csv_path : str
        Path to the CSV file containing data associated with each grid ID
    output_dir : str
        Directory where the output GeoJSON files will be saved
    id_field : str
        The field name used to join the datasets
    exclude_layers : list, optional
        List of layer names to exclude from processing
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize exclude_layers as empty list if None
    if exclude_layers is None:
        exclude_layers = []
    
    # Get all available layers
    available_layers = fiona.listlayers(grid_path)
    
    # Filter out excluded layers
    available_layers = [layer for layer in available_layers if layer not in exclude_layers]
    
    lcz_df = pd.read_csv(lcz_path)
    lcz_df = lcz_df[[id_field, 'City', 'LCZ']].copy()
    lcz_df = lcz_df.merge(lcz_label_df, left_on='LCZ', right_on='LCZ', how='left')

    # Read the CSV data once
    full_data_df = pd.read_csv(csv_path)
    full_data_df = full_data_df.merge(lcz_df, on=[id_field, 'City'], how='left')
    
    # Group layers by city
    geneva_layers = []
    zurich_layers = []
    
    for layer in available_layers:
        # Normalize layer name to lowercase for comparison
        layer_lower = layer.lower()
        if 'geneva' in layer_lower:
            geneva_layers.append(layer)
        elif 'zurich' in layer_lower:
            zurich_layers.append(layer)
    
    # Process Geneva layers
    if geneva_layers:
        # Filter CSV data for Geneva only
        geneva_data_df = full_data_df[full_data_df['City'].str.lower() == 'geneva'].copy()
        print(f"Filtered {len(geneva_data_df)} Geneva records from CSV data")
        
        process_city_layers(grid_path, geneva_data_df, geneva_layers, 
                           os.path.join(output_dir, "geneva_grid_data.geojson"), 
                           id_field, city_suffix="geneva")
    
    # Process Zurich layers
    if zurich_layers:
        # Filter CSV data for Zurich only
        zurich_data_df = full_data_df[full_data_df['City'].str.lower() == 'zurich'].copy()
        print(f"Filtered {len(zurich_data_df)} Zurich records from CSV data")
        
        process_city_layers(grid_path, zurich_data_df, zurich_layers, 
                           os.path.join(output_dir, "zurich_grid_data.geojson"), 
                           id_field, city_suffix="zurich")
    
    return geneva_layers, zurich_layers

def clean_layer_name(layer_name, city_suffix):
    """
    Extract the base layer name without city suffixes.
    
    Parameters:
    -----------
    layer_name : str
        Original layer name from GeoPackage
    city_suffix : str
        City suffix to remove (e.g., 'geneva', 'zurich')
    
    Returns:
    --------
    str
        Cleaned layer name without city-specific parts
    """
    # Convert to lowercase for case-insensitive matching
    layer_lower = layer_name.lower()
    city_suffix = city_suffix.lower()
    
    # Remove common prefixes or suffixes
    cleaned_name = layer_name
    
    # Pattern 1: Remove "_cityname"
    cleaned_name = re.sub(f"_{city_suffix}", "", cleaned_name, flags=re.IGNORECASE)

    # Pattern 2: Remove "-cityname"
    cleaned_name = re.sub(f"-{city_suffix}", "", cleaned_name, flags=re.IGNORECASE)

    return cleaned_name

def process_city_layers(grid_path, data_df, layers, output_path, id_field, city_suffix=""):
    """
    Process and merge multiple layers for a city into a single GeoJSON with one feature per grid cell.
    Dynamically uses the spatial join keys that are present in each layer.
    """
    print(f"Processing layers for {os.path.basename(output_path)}:")
    
    # All potential spatial properties to use for merging layers
    all_spatial_join_keys = ['id', 'left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry', 'typology']
    
    # Dictionary to store each layer's GeoDataFrame
    layer_gdfs = {}
    
    # Track the CRS for consistent projection
    common_crs = None
    
    # First, read all layers
    for layer in layers:
        print(f"  - Reading layer: {layer}")
        try:
            # Read the layer
            layer_gdf = gpd.read_file(grid_path, layer=layer)
            
            # Store CRS for later use
            if common_crs is None:
                common_crs = layer_gdf.crs
                
            # Clean the layer name to remove city suffix
            clean_name = clean_layer_name(layer, city_suffix)
            
            # Identify which spatial join keys are present in this layer
            available_keys = [key for key in all_spatial_join_keys if key in layer_gdf.columns]
            
            # Store the available keys with the layer
            layer_info = {
                'gdf': layer_gdf,
                'available_keys': available_keys
            }
            
            # Store in dictionary
            layer_gdfs[clean_name] = layer_info
            
        except Exception as e:
            print(f"    Error processing layer {layer}: {str(e)}")
    
    # If no layers were read successfully, return
    if not layer_gdfs:
        print(f"No layers could be processed for {os.path.basename(output_path)}")
        return
    
    # Start with the first layer as our base
    base_layer_name = list(layer_gdfs.keys())[0]
    combined_gdf = layer_gdfs[base_layer_name]['gdf'].copy()
    
    # Merge remaining layers one by one
    for layer_name, layer_info in list(layer_gdfs.items())[1:]:
        print(f"  - Merging layer: {layer_name}")
        
        # Get the layer GeoDataFrame
        layer_gdf = layer_info['gdf']
        
        # Determine which keys to use for merging with this layer
        # Find keys that exist in both the combined data and this layer
        available_keys = layer_info['available_keys']
        merge_keys = [key for key in available_keys if key in combined_gdf.columns]
        
        # Ensure we have at least some keys for joining, including 'geometry'
        if 'geometry' not in merge_keys:
            merge_keys.append('geometry')
        
        print(f"    Using merge keys: {merge_keys}")
        
        # Use outer join to keep all features
        combined_gdf = combined_gdf.merge(
            layer_gdf,
            on=merge_keys,
            how='outer'
        )
    
    # Add a column to track which city this is
    combined_gdf['City'] = city_suffix.lower()
    
    # Now join with the CSV data on id_field
    if id_field in combined_gdf.columns:
        print(f"  - Joining with CSV data on field: {id_field}")
        combined_gdf = combined_gdf.merge(data_df, on=id_field, how='left')
    else:
        print(f"  Warning: Combined data does not contain id_field '{id_field}', skipping CSV join")
    
    # Export to GeoJSON
    combined_gdf.to_file(output_path, driver="GeoJSON")
    print(f"Exported {len(combined_gdf)} unique grid cells to {output_path}")
    
    return combined_gdf

In [12]:

load_dotenv()

data_folder = os.getenv('CITYTHERM_DATA_FOLDER', str(Path.home() / "Data"))
grid_path = os.path.join(data_folder, "open data multidomain neighbourhood types and environmental quality.gpkg")
csv_path = os.path.join(data_folder, "open_data_neighbourhood_parameters.csv")
output_path = os.path.join(data_folder, "grid_data.geojson")
lcz_path = os.path.join(data_folder, "citytherm.csv")

In [13]:
# Create output directory
output_dir = os.path.join(data_folder, "processed")

# Process all layers by city
geneva_layers, zurich_layers = process_grid_data_by_city(
    grid_path=grid_path,
    csv_path=csv_path,
    lcz_path=lcz_path,
    output_dir=output_dir,
    id_field='id',
    exclude_layers=["LST-Geneva"]  # Exclude problematic layer
)

print("\nProcessed Geneva layers:")
for layer in geneva_layers:
    print(f"- {layer}")

print("\nProcessed Zurich layers:")
for layer in zurich_layers:
    print(f"- {layer}")

Filtered 241 Geneva records from CSV data
Processing layers for geneva_grid_data.geojson:
  - Reading layer: Air_quality-Geneva
  - Reading layer: cluster_geneva
  - Reading layer: Irradiance-Geneva
  - Reading layer: Noise-Geneva
  - Merging layer: cluster
    Using merge keys: ['id', 'left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry']
  - Merging layer: Irradiance
    Using merge keys: ['id', 'left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry', 'typology']
  - Merging layer: Noise
    Using merge keys: ['id', 'left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry']
  - Joining with CSV data on field: id
Exported 241 unique grid cells to /Users/hugosolleder/Library/CloudStorage/OneDrive-epfl.ch/Research IT/Advanced Services/0194 - CITYTHERM/Data/processed/geneva_grid_data.geojson
Filtered 1342 Zurich records from CSV data
Processing layers for zurich_grid_data.geojson:
  - Reading layer: Air_quality-Zurich
  - Reading layer: cluste

# Processing Building Data

We'll add functionality to append building geometries and heights from shapefiles to our per-city GeoJSON files. This will allow us to combine the grid data with building information for more comprehensive analysis.

In [14]:
def append_buildings_to_city_data(city_geojson_path, buildings_shapefile_path, output_path=None):
    """
    Append building geometries and heights from a shapefile to a city GeoJSON file.
    
    Parameters:
    -----------
    city_geojson_path : str
        Path to the city GeoJSON file containing grid data
    buildings_shapefile_path : str
        Path to the shapefile containing building geometries and heights
    output_path : str, optional
        Path where the combined output will be saved. If None, overwrites the input city_geojson_path.
    
    Returns:
    --------
    GeoDataFrame
        The combined GeoDataFrame with both grid and building data
    """
    # Use the original path if no output path is specified
    if output_path is None:
        output_path = city_geojson_path
    
    # Read the city GeoJSON data
    city_gdf = gpd.read_file(city_geojson_path)
    
    # Read the buildings shapefile
    try:
        print(f"Reading buildings from: {buildings_shapefile_path}")
        buildings_gdf = gpd.read_file(buildings_shapefile_path)
        
        # Check if height field exists in the buildings data
        height_field = "Height_med"
        # potential_height_fields = ['height', 'HEIGHT', 'Height', 'building_height', 'BUILDING_HEIGHT', 'h', 'H']
        # for field in potential_height_fields:
        #     if field in buildings_gdf.columns:
        #         height_field = field
        #         break
        
        # If no height field found, add a note
        if height_field is None:
            print("No standard height field found in buildings data. Available fields:")
            for col in buildings_gdf.columns:
                print(f"- {col}")
        else:
            print(f"Using height field: {height_field}")
        
        # Add a type field to distinguish grid cells from buildings
        city_gdf['feature_type'] = 'grid_cell'
        buildings_gdf['feature_type'] = 'building'
        
        # Ensure the CRS matches
        if city_gdf.crs != buildings_gdf.crs:
            print(f"Reprojecting buildings from {buildings_gdf.crs} to {city_gdf.crs}")
            buildings_gdf = buildings_gdf.to_crs(city_gdf.crs)
        
        # Combine the datasets
        # We're using concat rather than spatial join since we want to keep all geometries separate
        combined_gdf = pd.concat([city_gdf, buildings_gdf], ignore_index=True)
        
        # Save the combined data
        combined_gdf.to_file(output_path, driver="GeoJSON")
        print(f"Saved combined data with {len(city_gdf)} grid cells and {len(buildings_gdf)} buildings to {output_path}")
        
        return combined_gdf
    
    except Exception as e:
        print(f"Error processing buildings shapefile: {str(e)}")
        # Return the original city data if there's an error
        return city_gdf

In [15]:
append_buildings_to_city_data(
    city_geojson_path=os.path.join(output_dir, "geneva_grid_data.geojson"),
    buildings_shapefile_path=os.path.join(data_folder, "building height", "geneva_building_hight.shp"),
    output_path=os.path.join(output_dir, "geneva_grid_data_with_buildings.geojson")
)

append_buildings_to_city_data(
    city_geojson_path=os.path.join(output_dir, "zurich_grid_data.geojson"),
    buildings_shapefile_path=os.path.join(data_folder, "building height", "zurich_building_hight.shp"),
    output_path=os.path.join(output_dir, "zurich_grid_data_with_buildings.geojson")
)

Reading buildings from: /Users/hugosolleder/Library/CloudStorage/OneDrive-epfl.ch/Research IT/Advanced Services/0194 - CITYTHERM/Data/building height/geneva_building_hight.shp
Using height field: Height_med
Reprojecting buildings from COMPD_CS["CH1903+ / LV95 + LN02 height",PROJCS["CH1903+ / LV95",GEOGCS["CH1903+",DATUM["CH1903+",SPHEROID["Bessel 1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]],AUTHORITY["EPSG","6150"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4150"]],PROJECTION["Hotine_Oblique_Mercator_Azimuth_Center"],PARAMETER["latitude_of_center",46.9524055555556],PARAMETER["longitude_of_center",7.43958333333333],PARAMETER["azimuth",90],PARAMETER["rectified_grid_angle",90],PARAMETER["scale_factor",1],PARAMETER["false_easting",2600000],PARAMETER["false_northing",1200000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2056"]],VERT_CS["LN02 

,id,left,top,right,bottom,row_index,col_index,pm10_mean,pm25_mean,no2_mean,...,Length railway,Neighbourhood type,LCZ,lcz_code,description,color,geometry,feature_type,UUID,Height_med
0,178.0,2676974.694,1248306.233,2677224.694,1248056.233,24.0,3.0,12.034500,8.608157,13.186684,...,0.0,Z-D,11.0,LCZ 11 (A),Dense trees,#006A18,"MULTIPOLYGON (((2676974.694 1248306.233, 26772...",grid_cell,NaN,NaN
1,226.0,2677224.694,1249056.233,2677474.694,1248806.233,21.0,4.0,12.514815,8.840453,14.601169,...,0.0,Z-D,11.0,LCZ 11 (A),Dense trees,#006A18,"MULTIPOLYGON (((2677224.694 1249056.233, 26774...",grid_cell,NaN,NaN
2,227.0,2677224.694,1248806.233,2677474.694,1248556.233,22.0,4.0,12.369444,8.734144,14.112733,...,0.0,Z-D,11.0,LCZ 11 (A),Dense trees,#006A18,"MULTIPOLYGON (((2677224.694 1248806.233, 26774...",grid_cell,NaN,NaN
3,228.0,2677224.694,1248556.233,2677474.694,1248306.233,23.0,4.0,12.233222,8.682424,13.784554,...,0.0,Z-D,11.0,LCZ 11 (A),Dense trees,#006A18,"MULTIPOLYGON (((2677224.694 1248556.233, 26774...",grid_cell,NaN,NaN
4,229.0,2677224.694,1248306.233,2677474.694,1248056.233,24.0,4.0,12.145956,8.638159,13.479364,...,0.0,Z-D,11.0,LCZ 11 (A),Dense trees,#006A18,"MULTIPOLYGON (((2677224.694 1248306.233, 26774...",grid_cell,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON Z ((2684487.708 1246709.943 430.648, 2...",building,{59833994-E87A-44A1-BE21-2541B24C984A},8.866516
65144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON Z ((2684483.39 1246716.009 430.688, 26...",building,{33535292-C943-43F4-833C-DF02B93F07B3},8.732239
65145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON Z ((2684485.908 1246712.471 430.681, 2...",building,{4174C0D5-A124-482E-890C-FFD1BBA9BBF6},8.817261
65146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON Z ((2683622.732 1247056.744 423.586, 2...",building,{E8B62630-87BD-4957-A659-F787A0EF52F1},2.448868


In [ ]:
append_buildings_to_city_data(
    city_geojson_path=os.path.join(output_dir, "zurich_grid_data.geojson"),
    buildings_shapefile_path=os.path.join(data_folder, "building height", "zurich_building_hight.shp"),
    output_path=os.path.join(output_dir, "zurich_grid_data_with_buildings.geojson")
)

Reading buildings from: /Users/hugosolleder/Library/CloudStorage/OneDrive-epfl.ch/Research IT/Advanced Services/0194 - CITYTHERM/Data/building height/zurich_building_hight.shp
Using height field: Height_med
Reprojecting buildings from COMPD_CS["CH1903+ / LV95 + LN02 height",PROJCS["CH1903+ / LV95",GEOGCS["CH1903+",DATUM["CH1903+",SPHEROID["Bessel 1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]],AUTHORITY["EPSG","6150"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4150"]],PROJECTION["Hotine_Oblique_Mercator_Azimuth_Center"],PARAMETER["latitude_of_center",46.9524055555556],PARAMETER["longitude_of_center",7.43958333333333],PARAMETER["azimuth",90],PARAMETER["rectified_grid_angle",90],PARAMETER["scale_factor",1],PARAMETER["false_easting",2600000],PARAMETER["false_northing",1200000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2056"]],VERT_CS["LN02 